In [ ]:
import pandas as pd
from pandas import DataFrame

import lasio
import glob 

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')
%matplotlib inline
%load_ext autotime

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Get list of las files in the folder

path = 'Train Data set'
files = glob.glob(path + "/*.las")

In [ ]:
# Create well-wise dataframes and populate with logs from the las files

i = 1

for file in files:
    las = lasio.read(file)
    globals()[str("Well_"+str(i))] = las.df().reset_index().fillna(0)
    globals()[str("Well_"+str(i))]['Well Name'] = "Well_"+str(i)
    globals()[str("Well_"+str(i))]['Latitude'] = las.well.SLAT.value
    globals()[str("Well_"+str(i))]['Longitude'] = las.well.SLON.value
    i += 1

In [ ]:
# Create exploratory dataframe with surface locations and available logs to help us identify the inputs

wells = []
lat = []
lon = []
keys = []
logcount = []
units = []
uniqueunits = []
descrs = []
depth = []
i = 1

for file in files:
    well = "Well_"+str(i)
    las = lasio.read(file)
    latitude = las.well.SLAT.value
    longitude = las.well.SLON.value
    key = las.keys()
    md = las.well.STOP.value
    keycount = len(key)
    unit = []
    descr = []
    for j in range(keycount):
        unit.append(las.curves.items()[j][1].unit)
        descr.append(las.curves.items()[j][1].descr)
        j+=1
    i+=1
    uniqueunit = list(set(unit))
    lat.append(latitude)
    lon.append(longitude)
    depth.append(md)
    keys.append(key)
    logcount.append(keycount)
    wells.append(well)
    units.append(unit)
    uniqueunits.append(uniqueunit)
    descrs.append(descr)
    
welldf = DataFrame({'Wellname':wells,'Latitude':lat,'Longitude':lon,'Depth':depth, 'Log Count':logcount,'Log List':keys,
                    'Unit List':units, 'Unique Units':uniqueunits, 'Description': descrs})

In [ ]:
# Plot the lat,long and log count to get an idea on how the data is distributed

plt.figure(figsize=(8,8))
sns.scatterplot(x ='Latitude', y = 'Longitude', data = welldf, size= 'Log Count', sizes = (100,600))

# We see that there are two clusters of wells based on locations

In [ ]:
# plot a histogram to see the number of logs available in each 

plt.figure(figsize=(8,8))
sns.histplot(welldf['Log Count'], bins = 5)

In [ ]:
# Using the log list column in our exploratory DF
# We can try to figure out if the naming schemes of all the well logs is similar.

welldf_explode = welldf.explode('Log List')
welldf_explode['Units'] = list(welldf.explode('Unit List')['Unit List'])
welldf_explode['Description'] = list(welldf.explode('Description')['Description'])
welldf_explode.rename(columns = {'Log List':'Logs'}, inplace = True)
welldf_explode.drop(['Unit List'],axis = 1, inplace = True)

In [ ]:
# Plot the logs available based on mnemonics 
plt.figure(figsize=(40,7))
welldf_explode['Logs'].value_counts().plot.bar()
plt.title('Log Mnemonic Count in 234 Wells')

# We can see that the naming scheme is not really similar. 
# Only Depth and DTSM (target variable) are available for all the 234 wells.
# Other logs are either named differntly or unavailable. We might have to rename some of the wells and condition the dataset.

In [ ]:
# We can plot the number of logs based on units listed in the las files

plt.figure(figsize=(15,7))
welldf_explode['Units'].value_counts().plot.bar()
plt.title('Unit Count in 234 Wells')

# We can see that there are multiple logs with same units making the highest log count to 700 when the number of wells are 234

In [ ]:
# Lets plot the only the unique log units available in a well

plt.figure(figsize=(15,7))
welldf.explode('Unique Units')['Unique Units'].value_counts().plot.bar()
plt.title('Entire Well')

# Feature Selection and  Compound Feature Creation

Based on the log availability, we can select the following freatures initially

1. LAT, LONG: Relative location to include spatial variation
2. DEPT: Available for all the wells, might have to eliminate some laterals from the training set
3. DTSM: Available for all the wells, one well has DTSM with different units, check magnitude and eliminate if necessary
4. DTCO: Available for 223 Wells, we can impute the rest
5. RHOB: Available for 169 wells, some of them are density correction logs which needs to be removed and rest needs to be imputed
6. PEF: Available for 160 wells, rest need to be imputed (They are named differently, so we can create a new feature with an average value if more than one is available in a well)
7. GR: Available for 230 wells, an average GR feature needs to be created and imputed
8. NPHI: Available for 199 wells, an average NPHI feature needs to be created and imputed
9. RES: Available for 181 wells, but many different variations are available so depending on the magnitudes, we might have to eliminate the entire feature
10. CALI: Caliper logs are available for 187 wells but might not be effective in predicting target variables


In [ ]:
# Append the las files for all the wells into a single dataframe

well_logs = []
for i in range (1, len(wells)+1):
    well_logs.append(globals()["Well_"+str(i)])
    
welldf_entire = pd.concat(well_logs)

# Replacing zero values with NaNs
welldf_entire.replace(0,np.nan,inplace=True)

# Replacing negative values with NaNs
welldf_entire[welldf_entire.DTSM < 0] = np.nan

# Dropping the columns where DTSM(target variable) is null
welldf_entire.dropna(subset = ['DTSM'], inplace = True)

In [ ]:
# Based on the units, lets select the log mnemonics that we can create a compound feature

GAPI = list(set(welldf_explode[welldf_explode['Units'] == "GAPI"]['Logs'].value_counts().index))

# Remove sprectral gamma ray logs from the list of logs having units GAPI
SGR = ['SGRDD','HSGRD','HSGRS','SGRS','HSGR','SGR','HSGRR','MSGRR','SGRD','SGRDD','SGRR','HCGR']

for element in GAPI:
    if element in SGR:
        GAPI.remove(element)

DEC = list(set(welldf_explode[welldf_explode['Units'] == "DEC"]['Logs'].value_counts().index))

# One of the DTSM log has the units of DEC, removing it from the porosity logs
DEC.remove('DTSM')

IN = list(set(welldf_explode[welldf_explode['Units'] == "IN"]['Logs'].value_counts().index))

OHMM = list(set(welldf_explode[welldf_explode['Units'] == "OHMM"]['Logs'].value_counts().index))

GC3  = list(set(welldf_explode[welldf_explode['Units'] == "G/C3"]['Logs'].value_counts().index))

# Remove density correction logs from the list of logs having units G/C3
DCORR = ['HDRA','DRHO','DRH','ZCOR','DCOR','CORR','QRHO','QRHO_SLDT','QRHO']
for element in GC3:
    if element in DCORR:
        GC3.remove(element)
        
BE = list(set(welldf_explode[welldf_explode['Units'] == "B/E"]['Logs'].value_counts().index))

# MV = list(set(welldf_explode[welldf_explode['Units'] == "MV"]['Logs'].value_counts().index))
# LB = list(set(welldf_explode[welldf_explode['Units'] == "LB"]['Logs'].value_counts().index))
# MMHO = list(set(welldf_explode[welldf_explode['Units'] == "MMHO"]['Logs'].value_counts().index))

In [ ]:
# Take the row average of the features to create average features if more than one kind of log is available

welldf_entire['GR_average'] = welldf_entire[GAPI].mean(axis = 1)
welldf_entire['PHI_average'] = welldf_entire[DEC].mean(axis = 1)
welldf_entire['CALI_average'] = welldf_entire[IN].mean(axis = 1)
welldf_entire['RES_average'] = welldf_entire[OHMM].mean(axis = 1)
welldf_entire['RHOB_average'] = welldf_entire[GC3].mean(axis = 1)
welldf_entire['PEF_average'] = welldf_entire[BE].mean(axis = 1)

# welldf_entire['SP_average'] = welldf_entire[MV].mean(axis = 1)
# welldf_entire['Tension_average'] = welldf_entire[LB].mean(axis = 1)
# welldf_entire['Induction_average'] = welldf_entire[MMHO].mean(axis = 1)

In [ ]:
# Make a df with selected features

features_entire =  welldf_entire[['DEPT','Latitude','Longitude','DTCO','RHOB_average','PEF_average',
                                  'GR_average','PHI_average','CALI_average','RES_average', 'DTSM']]

In [ ]:
# Null check on final features selected, we will impute the points shown in yellow

plt.figure(figsize=(10,10))
sns.heatmap(features_entire.isnull(), cmap = 'plasma', yticklabels= False, cbar = False)

In [ ]:
# Replacing negative values with nans 
for column in features_entire.columns:
    features_entire[column].loc[features_entire[column] <= 0] = np.nan

# Model Building

In [ ]:
# Splitting the data into train and test for self-validation for all three dataframes

from sklearn.model_selection import train_test_split
X = features_entire.drop('DTSM', axis =1)
y = features_entire.DTSM

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
# As this is a regression problem, the magnitude distribution can sometimes affect the model prediction. 
# So it will be ideal to scale the features
# Then we can use the iterative imputer to fill the missing values of the scaled features
# Then finally you can put this through ML model, Random Forest regressor in this case

from sklearn.preprocessing import RobustScaler
from xgboost import XGBRegressor

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.metrics import mean_squared_error as MSE
from sklearn.pipeline import Pipeline

# define model pipeline

scaler = RobustScaler()
imputer = IterativeImputer(random_state= 42, missing_values = np.nan)
model = XGBRegressor(random_state = 42, n_jobs= -1, verbosity= 2)
pipeline = Pipeline(steps=[('s',scaler),('i', imputer), ('m', model)])

In [ ]:
# Fit the model to the training dataset
pipeline.fit(X_train, y_train)

# Make predictions with the test dataset
predictions = pipeline.predict(X_test)

# evaluate model by calculating the RMSE
rmse = np.sqrt(MSE(y_test, predictions))
rmse

# Hyper-Parameter Tuning for increased Accuracy

In [ ]:
# # Hyper parameter tuning in order to improve the accuracy with a wider net

# parameters = {'m__learning_rate': [.05, 0.1, .15],
#               'm__n_estimators': [500, 1000, 1500],
#               'm__max_depth': [5, 10, 20],
#               'm__min_child_weight': [5, 10],
#               'm__subsample': [0.7],
#               'm__colsample_bytree': [0.7]
#               }

# # Using grid search CV, we can test for wide parameter ranges and estimate the best settings.
# # VERY computationally expensive!

# from sklearn.model_selection import GridSearchCV

# grid = GridSearchCV(pipeline, parameters, scoring='neg_root_mean_squared_error', verbose=3, refit=True)
# grid.fit(X,y)

# best_params = grid.best_params_

In [ ]:
best_params

# Building a model using the tuned hyperparameters

In [ ]:
model = XGBRegressor(colsample_bytree = 0.7, learning_rate = 0.05, max_depth = 5,
                     min_child_weight = 10, n_estimators = 500, subsample = 0.7, 
                     random_state = 42, n_jobs = -1, verbosity = 2)

pipeline = Pipeline(steps=[('s',scaler),('i', imputer), ('m', model)])

# Test wells conditioning for the final leaderboard

In [ ]:
# Read the test well las files and save them as dataframes

test = 'All Test Data'
testfiles = glob.glob(test + "/*.las")

i =1
for testfile in testfiles:
    las = lasio.read(testfile)
    globals()[str("Test_Well_"+str(i))] = las.df().reset_index().fillna(0)
    globals()[str("Test_Well_"+str(i))]['Well Name'] = str(testfile)[14:-4]
    globals()[str("Test_Well_"+str(i))]['Latitude'] = las.well.SLAT.value
    globals()[str("Test_Well_"+str(i))]['Longitude'] = las.well.SLON.value
    i += 1


In [ ]:
# Create exploratory dataframe for test wells to check on the logs available

testwells = []
lat = []
lon = []
keys = []
logcount = []
units = []
uniqueunits = []
descrs = []
depth = []
i = 1

for testfile in testfiles:
    testwell = str(testfile)[14:-4]
    las = lasio.read(testfile)
    latitude = las.well.SLAT.value
    longitude = las.well.SLON.value
    key = las.keys()
    md = las.well.STOP.value
    keycount = len(key)
    unit = []
    descr = []
    for j in range(keycount):
        unit.append(las.curves.items()[j][1].unit)
        descr.append(las.curves.items()[j][1].descr)
        j+=1
    i+=1
    uniqueunit = list(set(unit))
    lat.append(latitude)
    lon.append(longitude)
    depth.append(md)
    keys.append(key)
    logcount.append(keycount)
    testwells.append(testwell)
    units.append(unit)
    uniqueunits.append(uniqueunit)
    descrs.append(descr)
    
test_welldf = DataFrame({'Wellname':testwells,'Latitude':lat,'Longitude':lon,'Depth':depth, 'Log Count':logcount,'Log List':keys,
                    'Unit List':units, 'Unique Units':uniqueunits, 'Description': descrs})

In [ ]:
# Plot the lat,long and log count to get an idea on how the data is distributed

plt.figure(figsize=(8,8))
sns.scatterplot(x ='Latitude', y = 'Longitude', data = welldf, size= 'Log Count', sizes = (100,600))
sns.scatterplot(x ='Latitude', y = 'Longitude', data = test_welldf, size= 'Log Count', sizes = (100,600),color = 'red')

# We can see that the relative locations of the test wells are well within the train dataset space

In [ ]:
# Using the log list column in our exploratory DF
# We can try to figure out if the naming schemes of all the test well logs is similar.

test_welldf_explode = test_welldf.explode('Log List')
test_welldf_explode['Units'] = list(test_welldf.explode('Unit List')['Unit List'])
test_welldf_explode['Description'] = list(test_welldf.explode('Description')['Description'])
test_welldf_explode.rename(columns = {'Log List':'Logs'}, inplace = True)
test_welldf_explode.drop(['Unit List'],axis = 1, inplace = True)

# Append the las files for all the wells into a single dataframe

test_well_logs = []
for i in range (1, len(testwells)+1):
    test_well_logs.append(globals()["Test_Well_"+str(i)])
    
test_welldf_entire = pd.concat(test_well_logs)


# Based on the units, lets select the log mnemonics that we can create a compound feature

GAPI = list(set(test_welldf_explode[test_welldf_explode['Units'] == "GAPI"]['Logs'].value_counts().index))

# Remove sprectral gamma ray logs from the list of logs having units GAPI
SGR = ['SGRDD','HSGRD','HSGRS','SGRS','HSGR','SGR','HSGRR','MSGRR','SGRD','SGRDD','SGRR','HCGR']

for element in GAPI:
    if element in SGR:
        GAPI.remove(element)

DEC = list(set(test_welldf_explode[test_welldf_explode['Units'] == "DEC"]['Logs'].value_counts().index))

IN = list(set(test_welldf_explode[test_welldf_explode['Units'] == "IN"]['Logs'].value_counts().index))

OHMM = list(set(test_welldf_explode[test_welldf_explode['Units'] == "OHMM"]['Logs'].value_counts().index))

GC3  = list(set(test_welldf_explode[test_welldf_explode['Units'] == "G/C3"]['Logs'].value_counts().index))

# Remove density correction logs from the list of logs having units G/C3
DCORR = ['HDRA','DRHO','DRH','ZCOR','DCOR','CORR','QRHO','QRHO_SLDT','QRHO']
for element in GC3:
    if element in DCORR:
        GC3.remove(element)
        
BE = list(set(test_welldf_explode[test_welldf_explode['Units'] == "B/E"]['Logs'].value_counts().index))

# Take the row average of the features to create average features if more than one kind of log is available

test_welldf_entire['GR_average'] = test_welldf_entire[GAPI].mean(axis = 1)
test_welldf_entire['PHI_average'] = test_welldf_entire[DEC].mean(axis = 1)
test_welldf_entire['CALI_average'] = test_welldf_entire[IN].mean(axis = 1)
test_welldf_entire['RES_average'] = test_welldf_entire[OHMM].mean(axis = 1)
test_welldf_entire['RHOB_average'] = test_welldf_entire[GC3].mean(axis = 1)
test_welldf_entire['PEF_average'] = test_welldf_entire[BE].mean(axis = 1)

test_features =  test_welldf_entire[['DEPT','Latitude','Longitude','DTCO','RHOB_average','PEF_average',
                                  'GR_average','PHI_average','CALI_average','RES_average']]

# fill null values as zeroes
test_features.fillna(0, axis =1, inplace = True)

# Change negative values to zeroes
test_features[test_features < 0] = 0

# Creating a submission file

In [ ]:
# Fit the data too the pipeline using the complete dataset
pipeline.fit(X, y)

# Predict the DTSM for test Wells

test_predictions = pipeline.predict(test_features)

# Make a Dataframe of predictions of test dataset
submission_file = test_welldf_entire[['Well Name','DEPT']]
submission_file['DTSM'] = test_predictions

In [ ]:
# Plot feature Importance and see if any insignificant features can be eliminated

feat_imp = DataFrame(X.columns, columns=['Feature'])
feat_imp['Feature Importance'] = pipeline.steps[2][1].feature_importances_

feat_imp.sort_values(by = 'Feature Importance', ascending=False, inplace = True)

plt.figure(figsize=(8,8))
sns.barplot(x = 'Feature Importance', y = 'Feature', data = feat_imp)

In [ ]:
# Checking the cross-validation scores
scores = cross_validate(pipeline, X, y, cv = 10, scoring = ('neg_root_mean_squared_error'))
print("RMSE for xGBoost Regression: ",np.average(scores.get('test_score')))

In [ ]:
# Exporting the predicted well logs to their xlsx file
for testwell in testwells:
    submission_file[submission_file['Well Name'] == str(testwell)].drop('Well Name', axis = 1).to_excel(str(testwell)+'.xlsx', index = False)
submission_file.to_csv('xgboost_tuned_new.csv')

# Saving all the conditioned data to save time and re-using a saved model

In [ ]:
# Save conditioned and cleaned X and Y of bothe train and test datasets for easy reuse
features_entire.to_csv('Train_Set_Conditioned.txt', index = False)
test_features.to_csv('All_Test_Set_Conditioned.txt', index = False)

import pickle

# Save to file in the current working directory
pkl_filename = "xgboost_model_tuned_new.pkl"


with open(pkl_filename, 'wb') as file:
    pickle.dump(pipeline, file)

In [ ]:
# Load from file
pkl_filename = "xgboost_model_tuned_new.pkl"

with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)
    
# Make submission file

test_predictions = pickle_model.predict(test_features)
submission_file = test_welldf_entire[['Well Name','DEPT']]
submission_file['DTSM'] = test_predictions

# Exporting the predicted well logs to their xlsx file
for testwell in testwells:
    submission_file[submission_file['Well Name'] == str(testwell)].drop('Well Name', axis = 1).to_excel(str(testwell)+'.xlsx', index = False)
submission_file.to_csv('xgboost_tuned_new.csv')